In [1]:
import cv2
import numpy as np
import os
from random import shuffle
from tqdm import tqdm
import glob
TRAIN_DIR = 'E:/University/Project/train'
TEST_DIR = 'E:/University/Project/testwithlabel'
LR = 1e-3

#MODEL_NAME = 'model_2_layers_withoutdropout_32'


In [2]:
def label_img(img):
    word_label = img.split('.')[-3]
    if word_label == 'cat':
        return [1, 0]
    elif word_label == 'dog':
        return [0, 1]


In [3]:
def create_train_data(fname,img_size):
    training_data = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        label = label_img(img)
        path = os.path.join(TRAIN_DIR, img)
        img = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE), (img_size, img_size))
        training_data.append([np.array(img)/255., np.array(label)])
    shuffle(training_data)
    np.save(fname + '.npy', training_data)
    return training_data


In [4]:
def create_test_data(fname,img_size):
    testing_data = []
    for img in tqdm(os.listdir(TEST_DIR)):
        path = os.path.join(TEST_DIR, img)
        img_num = img.split('.')[0]
        img = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE), (img_size, img_size))
        testing_data.append([np.array(img)/255, img_num])

    np.save(fname + '.npy', testing_data)
    return testing_data

In [5]:
#IMG_SIZE =  [64]

#for imgsize in IMG_SIZE:
    #fname = "Training_"+str(imgsize)
    #create_train_data(fname,imgsize)
    #fname = "Testing_"+str(imgsize)
    #create_test_data(fname,imgsize)


100%|███████████████████████████████████| 25000/25000 [01:33<00:00, 267.32it/s]


In [ ]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

import tensorflow as tf
tf.reset_default_graph()

In [ ]:
def modelgen(layer, drop=False):
    convnet = input_data(shape=[None, IMG_SIZEE, IMG_SIZEE, 1], name='input')
    if layer < 3:
        convnet = conv_2d(convnet, 32, 3, activation='relu')
        convnet = max_pool_2d(convnet, 2)
        convnet = conv_2d(convnet, 64, 3, activation='relu')
        convnet = max_pool_2d(convnet, 2)
    if layer < 4:
        convnet = conv_2d(convnet, 128, 3, activation='relu')
        convnet = max_pool_2d(convnet, 2)
    if layer < 5:
        convnet = conv_2d(convnet, 256, 3, activation='relu')
        convnet = max_pool_2d(convnet, 2)
    convnet = fully_connected(convnet, 1024, activation='relu')
    if drop:
        convnet = dropout(convnet, 0.8)
    convnet = fully_connected(convnet, 2, activation='softmax')
    convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')
    model = tflearn.DNN(convnet, tensorboard_dir='log')
    return model

In [ ]:
layers = [2] #(changeble between 2,3)
listtraining = ['Training_32.npy'] #((changeble between 32,64)
dropoutflag = [True] #(changeble between true, false)
for trainnpy in listtraining:
    testnpy = trainnpy.replace('Training_','Testing_')
    IMG_SIZEE = int(trainnpy.replace('.npy', '').replace('Training_', ''))
    train = []
    test = []
    train_data = np.load('Training_32.npy', allow_pickle=True)
    # test = np.load(testnpy, allow_pickle=True)
    train = train_data[:-1000]
    test = train_data[-1000:]
    X = 0
    Y = 0
    X = np.array([i[0] for i in train]).reshape(-1, IMG_SIZEE, IMG_SIZEE, 1)
    Y = [i[1] for i in train]
    test_x = 0 
    test_y = 0
    test_x = np.array([i[0] for i in test]).reshape(-1, IMG_SIZEE, IMG_SIZEE, 1)
    test_y = [i[1] for i in test]
    for ll in layers: # 3 sets of models
        for dd in dropoutflag: # 2 sets of dropout
            MODEL_NAME = trainnpy.replace('.npy','').replace('Training_','')
            if dd:
                MODEL_NAME = "model_" + str(ll) + '_layers_' + "withdropout_" + MODEL_NAME
            else:
                MODEL_NAME = "model_" + str(ll) + '_layers_' + "withoutdropout_" + MODEL_NAME
            model = False
            model= modelgen(ll, dd)
            model.fit({'input': X}, {'targets': Y}, n_epoch=30,validation_set=({'input': test_x}, {'targets': test_y}),
                      snapshot_step=500, show_metric=True, run_id=MODEL_NAME)
            model.save(MODEL_NAME)

In [ ]:
IMG_SIZEE = [32, 64]
imgsize = IMG_SIZEE
VAL_DIR = 'E:/University/Project/testwithlabel'
fname = "TestingWithLabel_"+str(imgsize)
create_validation_data(fname,IMG_SIZEE)
fname

In [ ]:
IMG_SIZEE = 64
model = modelgen(2,True)

In [ ]:
test_data = np.load('TestingWithLabel_64.npy', allow_pickle=True)
#train = test_data[:-1000]
#test = test_data[-1000:]
X = 0
Y = 0
X = np.array([i[0] for i in test_data]).reshape(-1, IMG_SIZEE, IMG_SIZEE, 1)
Y = [i[1] for i in test_data]

In [ ]:
model_name = 'model_2_layers_withdropout_64'
model.load(model_name)

In [ ]:
y_pred = model.predict(X)

In [ ]:
y_classes = y_pred.argmax(axis=-1)

In [ ]:
newY= np.array(Y)
newY[:5]

In [ ]:
y_true = newY.argmax(axis=-1)

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score 
print("=======>",model_name, "<=======")
print(classification_report(y_true, y_classes, labels=[1,0]))
print('accuracy_score = ',accuracy_score(y_true, y_classes))